# RPi Robot Hat library
**Full Coding At [Ultrasonic_sens.py](Ultrasonic_sens.py)** 
- *Library Used* 
    - RPi_GPIO   
    - time 

## Installing Libraries (Dependencies)
- RPi.GPIO 
    - `pip install rpi-lgpio` 

## Let's Start Coding ! 
### 1. Import all the libraries 
- RPi.GPIO 
- time 

In [ ]:
import RPi.GPIO as GPIO
import time

### 2. Create the Ultrasonic Class
**As this Script will be a Library for other script thus a class is needed* 
- `__init_check` <br> 
    Is used to make sure the library is not being initialise multiple time 
- `SOUND_SPEED` <br>
    Is the speed of sound in cm/s. In default the value is *34300*

In [ ]:
class Ultrasonic:
    """
    Class to represent an ultrasonic sensor.
    """
    __init_check = False 
    SOUND_SPEED = 34300  # Speed of sound in cm/s

### 3. The `__init__` function 
- The `__init__` function will run when the code is initialise 
- This function is used to intiakise the sensor and also GPIO Pins. 
- This function will also holds and intialise a few parameters.
    - `Left_sensor`: The Pin number for the Left Ultrasonic sensor 
    - `Front_sensor`: The Pin number for the front Ultrasonic sensor
    - `Right_sensor`: The Pin Number for the Right ultrasonic sensor 
    - `debug`: The debug flag. **False** in default. 
- When the function is called, it will check the initialisation state first. 
    ```python 
    if not Ultrasonic.__init_check:
    ```
    - Initialation process will start if the `__init_check` is **False** 
        - Set the GPIO mode to BCM Mode 
            ```python 
            GPIO.setmode(GPIO.BCM)
            ```
        - Set the GPIO Pin to OUTPUT mode 
            ```python
            GPIO.setup(Left_sensor, GPIO.OUT)
            GPIO.setup(Front_sensor, GPIO.OUT)
            GPIO.setup(Right_sensor, GPIO.OUT)
            ```
        - Set initialation state to **True**
    - If the `__init_check` is **True**, it will skip the initialation proces

In [ ]:
def __init__(self, Left_sensor=5, Front_sensor=16, Right_sensor = 18, debug=False):
        """
        Initializes GPIO pins for the ultrasonic sensors.
        :param Left_sensor: Left sensor GPIO pin (default 5)
        :param Front_sensor: Front sensor GPIO pin (default 16)
        :param Right_sensor: Right sensor GPIO pin (default 18)
        :param debug: Enable debug mode (default False)
        """
        if not Ultrasonic.__init_check:
            
            self.Left_sensor = Left_sensor
            self.Front_sensor = Front_sensor  
            self.Right_sensor = Right_sensor
            self.debug = debug

            GPIO.setmode(GPIO.BCM)
            GPIO.setup(Left_sensor, GPIO.OUT)
            GPIO.setup(Front_sensor, GPIO.OUT)
            GPIO.setup(Right_sensor, GPIO.OUT)
            if self.debug:
                print("Ultrasonic sensor initialized.")
            Ultrasonic.__init_check = True
        else:
            if self.debug: 
                print("Ultrasonic sensor already initialized.")
            pass

### 4. The `send_trigger_pulse` function 
- This function will trigger the Ultrasonic Ranger to send a pulse 
- To trigger the ultrasonic ranger, Triger the signal pin to True (HIGH)
    ```python 
    GPIO.output(pin, True)
    ```
- Wait for 10 milisecond 
    ```python 
    time.sleep(0.001)
    ```
- End the trigger by setting the GPIO Out to False (LOW)
    ```python
    GPIO.output(pin, False)
    ```

In [ ]:
    def send_trigger_pulse(self, pin):
        """
        Sends a 10 microsecond high pulse on the specified pin to trigger the sensor.
        """
        GPIO.output(pin, True)
        time.sleep(0.001) 
        GPIO.output(pin, False)
        if self.debug:
            print(f"Trigger Sent{pin}")


### 5. The `wait_for_echo` function 
- This function is used to recive the echo send by the ultrasonic 
- The function will set the GPIO pin to INPUT mode
    ```python 
    GPIO.setup(pin, GPIO.IN)
    ``` 
- Set the timeout duration 
    ```python
    timeout = 0.01
    ```

- Obtain the starting time 
    ```python
    'Initial_Time' = time.time()
    ``` 
- Detect is the signal recived by the sensor 
    - When signal is not recived, the GPIO is 0 
    
        ```python
        while GPIO.input(pin) == 0:
            if time.time() - Initial_Time >= timeout:
                if self.debug:
                    print("Time Out: No rising edge detected.")
                return None  
        ```
    - Obtain the pulse start time 
        ```python 
        pulse_start = time.time()
        ```
    - When the pin recive a signal, Obtain the end time 
        ```python 
        while GPIO.input(pin) == 1:
         pulse_end = time.time()
         if self.debug:
             print("Echo received")
        ```

    - Calculate the pulse duration
        - The duration can be calculated using the formula **Pulse_End - Pulse_Star = Pulse_Duration** 
        ```python
        pulse_duration = pulse_end - pulse_start
        ```
    
    - Return the pulse_duration as the result 
        ```python
        return pulse_duration
        ```

In [ ]:
def wait_for_echo(self, pin):
        """
        Waits for a rising edge on the echo pin (specified pin) and measures the pulse duration.
        """
        GPIO.setup(pin, GPIO.IN)
        timeout = 0.01  # Timeout set to 10 milliseconds
        
        Initial_Time = time.time()
        
        # Wait for the rising edge
        while GPIO.input(pin) == 0:
            if time.time() - Initial_Time >= timeout:
                if self.debug:
                    print("Time Out: No rising edge detected.")
                return None  # Return None if no rising edge is detected within timeout
        
       
        pulse_start = time.time()
        # Measure the pulse duration
        while GPIO.input(pin) == 1:
            pulse_end = time.time()
            if self.debug:
                print("Echo received")
        
        # Calculate and return the pulse duration
        pulse_duration = pulse_end - pulse_start
        return pulse_duration


### 6. The `get_distance` function 
- This fucntion will calculate the distance from the obstacle 
- Set the pin to OUTPUT to send the trigger pulse 
    ```python 
    GPIO.setup(pin, GPIO.OUT)
    self.send_trigger_pulse(pin)
    ```
- Get the pulse duration from the `wait_for_echo` function  
    ```python
    pulse_duration = self.wait_for_echo(pin)
    ```
- Makesure the value of the pulse_duration is not **None** 
    - If the pulse_duration is  **None**
    - Send the pulse again 
        ```python
        if pulse_duration is None:
                GPIO.setup(pin, GPIO.OUT)
                self.send_trigger_pulse(pin)
                print("Error: No echo received. Retrying...")
                time.sleep(0.1)
                self.send_trigger_pulse(pin)  # Retry sending trigger pulse
                pulse_duration = self.wait_for_echo(pin)  # Wait for echo again
                return None 
        ```
    - Else (If the pulse_duration is **NOT None**) 
    - Calculate the distance using the formula 
        $$
        distance = \frac {(pulse duration * Speed of sound)} {2} 
        $$
        ```python
        else: 
            distance = (pulse_duration * self.SOUND_SPEED) / 2
        ```


In [ ]:
    def get_distance(self, pin):
        """
        Measures distance using the ultrasonic sensor connected to the specified pin and returns the calculated value.
        """
        GPIO.setup(pin, GPIO.OUT)
        self.send_trigger_pulse(pin)

        pulse_duration = self.wait_for_echo(pin)

        if pulse_duration is None:
            GPIO.setup(pin, GPIO.OUT)
            self.send_trigger_pulse(pin)
            print("Error: No echo received. Retrying...")
            time.sleep(0.1)
            self.send_trigger_pulse(pin)  # Retry sending trigger pulse
            pulse_duration = self.wait_for_echo(pin)  # Wait for echo again
            return None 
        else: 
            distance = (pulse_duration * self.SOUND_SPEED) / 2

        if self.debug:
            print(f"Distance: {distance} cm for pin {pin}")
        return distance

### 7. The `distances` function 
- This function will obtain all the distance from the Left, Front and Right sensor 
- The value of all the sensor is obtained uisng the `get_distance` function 
- The result will be return with the sequence 
    - **Left, Front, Right**  

In [ ]:
def distances(self):
        """
        Get the distance measurement form the left, front and right sensor 
        """
        Left = self.get_distance(self.Left_sensor)
        Front = self.get_distance(self.Front_sensor)
        Right = self.get_distance(self.Right_sensor)
        if self.debug:
            print(f"Left: {Left}, Front: {Front}, Right:{Right}")
        return Left, Front, Right

### 8. The `cleanup` function 
- This function is used at the end of the code 
- It will release all the Pins and unassigned them 


In [ ]:
    def cleanup(self):
        """
        Cleans up GPIO pins
        """
        if self.debug:
            print("Cleaning up GPIO pins...")
        GPIO.cleanup()

### 9. The Test Code 
- If this script is run individually, the code bellow will start 
- This is also an example to how to use this Library 
<br>
<br>
****To Test the code Run the [Ultrasonic_sens.py](Ultrasonic_sens.py)***

In [ ]:
if __name__ == "__main__":
    try:
        ultrasonic = Ultrasonic(debug=True)
        while True: 
            Left, Front, Right = ultrasonic.distances()
            if (Left and Front and Right) is not None: 
                print("Left: {:.2f} cm".format(Left))
                print("Front: {:.2f} cm".format(Front))
                print("Right: {:.2f} cm".format(Right))
                time.sleep(1)
                print(" ")
    except KeyboardInterrupt:
        ultrasonic.cleanup()
        print("Exiting...")